# Mas APIS DE EQUITY

# Ameritrade

Para acceder a la API de TDA el broker ameritrade, hay que ingresar a https://developer.tdameritrade.com/
    
Y obviamente sacarse una cuenta de usuario gratuita, yo saqué la mia

Una vez creada la cuenta tienen que crear una app para que le den una "Consumer Key" similar a esta:

In [10]:
c_key = 'AHSA7C8FWZZFRG1W15WOJLQY8WRSCOVI'

Una vez obtenida la key pueden usarla para llamar a la API

Obviamente hay que importar requests

In [11]:
import requests

Y ahora si probamos algún endpoint

In [20]:
def getQuote(symbol):
    
    endpoint = 'https://api.tdameritrade.com/v1/marketdata/'+symbol+'/quotes?'
    
    p  ={'apikey' : c_key}
    r = requests.get(url=endpoint ,params=p)
    return r.json()[symbol]

quote = getQuote('AAPL')
quote

{'assetType': 'EQUITY',
 'assetMainType': 'EQUITY',
 'cusip': '037833100',
 'symbol': 'AAPL',
 'description': 'Apple Inc. - Common Stock',
 'bidPrice': 112.56,
 'bidSize': 200,
 'bidId': 'K',
 'askPrice': 112.6,
 'askSize': 200,
 'askId': 'Z',
 'lastPrice': 112.59,
 'lastSize': 0,
 'lastId': 'K',
 'openPrice': 115.23,
 'highPrice': 116.0,
 'lowPrice': 112.04,
 'bidTick': ' ',
 'closePrice': 115.54,
 'netChange': -2.95,
 'totalVolume': 154527505,
 'quoteTimeInLong': 1600292332694,
 'tradeTimeInLong': 1600292334840,
 'mark': 112.56,
 'exchange': 'q',
 'exchangeName': 'NASD',
 'marginable': True,
 'shortable': True,
 'volatility': 0.0206,
 'digits': 4,
 '52WkHigh': 137.98,
 '52WkLow': 53.1525,
 'nAV': 0.0,
 'peRatio': 35.0697,
 'divAmount': 0.82,
 'divYield': 0.71,
 'divDate': '2020-08-07 00:00:00.000',
 'securityStatus': 'Normal',
 'regularMarketLastPrice': 112.13,
 'regularMarketLastSize': 1,
 'regularMarketNetChange': -3.41,
 'regularMarketTradeTimeInLong': 1600286401169,
 'netPercentC

In [24]:
def getQuotes(symbols):
    
    endpoint = 'https://api.tdameritrade.com/v1/marketdata/quotes'
    r = requests.get(url=endpoint, params={'apikey' : c_key, 'symbol':symbols})
    
    return r.json()

quotes = getQuotes('AAPL,YPF')

quotes.keys()

dict_keys(['AAPL', 'YPF'])

In [23]:
quotes['AAPL']

{'assetType': 'EQUITY',
 'assetMainType': 'EQUITY',
 'cusip': '037833100',
 'symbol': 'AAPL',
 'description': 'Apple Inc. - Common Stock',
 'bidPrice': 112.6,
 'bidSize': 3500,
 'bidId': 'Z',
 'askPrice': 112.63,
 'askSize': 300,
 'askId': 'P',
 'lastPrice': 112.62,
 'lastSize': 0,
 'lastId': 'K',
 'openPrice': 115.23,
 'highPrice': 116.0,
 'lowPrice': 112.04,
 'bidTick': ' ',
 'closePrice': 115.54,
 'netChange': -2.92,
 'totalVolume': 154558774,
 'quoteTimeInLong': 1600292822172,
 'tradeTimeInLong': 1600292826646,
 'mark': 112.6,
 'exchange': 'q',
 'exchangeName': 'NASD',
 'marginable': True,
 'shortable': True,
 'volatility': 0.0205,
 'digits': 4,
 '52WkHigh': 137.98,
 '52WkLow': 53.1525,
 'nAV': 0.0,
 'peRatio': 35.0697,
 'divAmount': 0.82,
 'divYield': 0.71,
 'divDate': '2020-08-07 00:00:00.000',
 'securityStatus': 'Normal',
 'regularMarketLastPrice': 112.13,
 'regularMarketLastSize': 1,
 'regularMarketNetChange': -3.41,
 'regularMarketTradeTimeInLong': 1600286401169,
 'netPercentC

### Precios históricos

* Periodos válidos para cada periodType (los del * son los default)
    * day: 1, 2, 3, 4, 5, 10*
    * month: 1*, 2, 3, 6
    * year: 1*, 2, 3, 5, 10, 15, 20
    * ytd: 1*

* FrequencyType válidos para cada periodType (los del * son los default)
    * day: minute*
    * month: daily, weekly*
    * year: daily, weekly, monthly*
    * ytd: daily, weekly*
    
* Frequency válidos para cada FrequencyType (los del * son los default)
    * minute: 1*, 5, 10, 15, 30
    * daily: 1*
    * weekly: 1*
    * monthly: 1*

Otros parámetros posibles:
    * startDate: timestamp milisegundos desde epoch
    * endDate: timestamp milisegundos desde epoch

In [35]:
import pandas as pd

def getHist(symbol, periodT='day', period=1, freqT='minute', freq=1, na='true'):
    
    
    params = {'symbol':symbol, 'periodType':periodT, 'period':period, 
              'apikey' : c_key, 'frequencyType':freqT, 'frequency':freq ,
              'needExtendedHoursData':na}
    
    url_base='https://api.tdameritrade.com/v1/marketdata/'
    endpoint = url_base + symbol+'/pricehistory'
    
    r = requests.get(url=endpoint ,params=params)
    try:
        js = r.json()['candles']
        df = pd.DataFrame(js)
        df['fecha'] = pd.to_datetime(df.datetime - 3600*1000*3, unit='ms')
    except:
        print('Error, ojo que tenemos una rta inesperada', r.json())
        df = pd.DataFrame()
        
    return df

data = getHist('GGAL', periodT='day', period=1, freqT='minute', freq=1, na='false')
data

,open,high,low,close,volume,datetime,fecha
0,9.2200,9.2700,9.220,9.2700,4308,1600176600000,2020-09-15 10:30:00
1,9.2550,9.2899,9.255,9.2688,4900,1600176660000,2020-09-15 10:31:00
2,9.2450,9.2450,9.245,9.2450,100,1600176720000,2020-09-15 10:32:00
3,9.2500,9.2500,9.170,9.1700,2501,1600176780000,2020-09-15 10:33:00
4,9.1800,9.2156,9.180,9.2100,1000,1600176840000,2020-09-15 10:34:00
...,...,...,...,...,...,...,...
342,9.1900,9.1900,9.180,9.1800,2334,1600199700000,2020-09-15 16:55:00
343,9.1700,9.1750,9.160,9.1650,3872,1600199760000,2020-09-15 16:56:00
344,9.1695,9.1800,9.160,9.1700,5205,1600199820000,2020-09-15 16:57:00
345,9.1700,9.1750,9.160,9.1600,2554,1600199880000,2020-09-15 16:58:00


Como verán en este caso usamos un try/except por si pasamos mal algun parámetro pero que ademas del mensaje de error me devuelva igual el dataFrame vacio

Esto es una buena práctica ppara evitar cortes del flujo del script inesperados

### Fundamentals

In [6]:
def fundamentals(symbol, pr='fundamental'):
    
    params = {'apikey' : c_key, 'symbol':symbol, 'projection': pr}
    
    endpoint = 'https://api.tdameritrade.com/v1/instruments'
    
    r = requests.get(url=endpoint ,params=params)
    return r.json()

fundamentals('AAPL')

{'AAPL': {'fundamental': {'symbol': 'AAPL',
   'high52': 399.82,
   'low52': 192.58,
   'dividendAmount': 3.28,
   'dividendYield': 0.84,
   'dividendDate': '2020-05-08 00:00:00.000',
   'peRatio': 30.50251,
   'pegRatio': 3.268477,
   'pbRatio': 21.4526,
   'prRatio': 6.29316,
   'pcfRatio': 24.46643,
   'grossMarginTTM': 38.10979,
   'grossMarginMRQ': 38.36194,
   'netProfitMarginTTM': 21.35039,
   'netProfitMarginMRQ': 19.29072,
   'operatingMarginTTM': 24.47599,
   'operatingMarginMRQ': 22.0414,
   'returnOnEquity': 62.09404,
   'returnOnAssets': 17.27511,
   'returnOnInvestment': 24.21635,
   'quickRatio': 1.46127,
   'currentRatio': 1.49596,
   'interestCoverage': 0.0,
   'totalDebtToCapital': 58.41288,
   'ltDebtToEquity': 114.3959,
   'totalDebtToEquity': 140.459,
   'epsTTM': 12.756,
   'epsChangePercentTTM': 9.33233,
   'epsChangeYear': 3.83907,
   'epsChange': 0.0,
   'revChangeYear': 0.0,
   'revChangeTTM': 3.67171,
   'revChangeIn': 0.0,
   'sharesOutstanding': 4334335000.

### Cadena de Opciones

Este es el endpoint mas jugoso de la API a mi entender ya que es un dato de mercado dificil de conseguir en  formma gratuita y fidedigna esto de las cadenas de opciones

Les dejo la implementación en python de un llamado al endpoint de opciones sencillo que me devuelve en principio todo un json y como es muy compleja la estructura vamos trabajandola de a poco luego

In [38]:
import requests

def options(symbol):
    
    params = {'apikey' : c_key, 'symbol':symbol}
    
    endpoint = 'https://api.tdameritrade.com/v1/marketdata/chains'
    
    r = requests.get(url=endpoint ,params=params)
    return r.json()

chain = options('GGAL')
chain

{'symbol': 'GGAL',
 'status': 'SUCCESS',
 'underlying': None,
 'strategy': 'SINGLE',
 'interval': 0.0,
 'isDelayed': True,
 'isIndex': False,
 'interestRate': 0.1,
 'underlyingPrice': 8.355,
 'volatility': 29.0,
 'daysToExpiration': 0.0,
 'numberOfContracts': 118,
 'putExpDateMap': {'2020-09-18:2': {'2.5': [{'putCall': 'PUT',
     'symbol': 'GGAL_091820P2.5',
     'description': 'GGAL Sep 18 2020 2.5 Put',
     'exchangeName': 'OPR',
     'bid': 0.0,
     'ask': 0.05,
     'last': 0.0,
     'mark': 0.03,
     'bidSize': 0,
     'askSize': 100,
     'bidAskSize': '0X100',
     'lastSize': 0,
     'highPrice': 0.0,
     'lowPrice': 0.0,
     'openPrice': 0.0,
     'closePrice': 0.0,
     'totalVolume': 0,
     'tradeDate': None,
     'tradeTimeInLong': 0,
     'quoteTimeInLong': 1600285790589,
     'netChange': 0.0,
     'volatility': 676.61,
     'delta': -0.011,
     'gamma': 0.006,
     'theta': -0.026,
     'vega': 0.0,
     'rho': 0.0,
     'openInterest': 0,
     'timeValue': 0.03,

Lo que me devuelve es una estructura mas o menos asi:


{'symbol': 'GGAL',<br>
 'status': 'SUCCESS',<br>
 'underlying': None,<br>
 'strategy': 'SINGLE',<br>
 'interval': 0.0,<br>
 'isDelayed': True,<br>
 'isIndex': False,<br>
 'interestRate': 0.1,<br>
 'underlyingPrice': 11.41,<br>
 'volatility': 29.0,<br>
 'daysToExpiration': 0.0,<br>
 'numberOfContracts': 120,<br>
 'putExpDateMap': {'2020-07-17:0': {'2.5': [.. y aca todas las claves de ese strike de ese vencimiento de ese side(put)
 
 ... y asi los (en este caso 60) puts, y luego lo mismo para los 60 calls

Como esa estructura es demasiado rebuscada para listar, calcular, graficar, etc, vamos mejor a crear listados de fechas de vencimientos y de contratos por un lado y luego vamos a ir metiendolo en un dataFrame

In [39]:
v_calls = list(chain['callExpDateMap'].values())
v_calls_fechas = list(chain['callExpDateMap'].keys())
v_puts = list(chain['putExpDateMap'].values())
v_puts_fechas = list(chain['putExpDateMap'].keys())

print(v_calls_fechas)

['2020-09-18:2', '2020-10-16:30', '2021-01-15:121', '2021-04-16:212', '2022-01-21:492', '2023-01-20:856']


In [41]:
v_puts

[{'2.5': [{'putCall': 'PUT',
    'symbol': 'GGAL_091820P2.5',
    'description': 'GGAL Sep 18 2020 2.5 Put',
    'exchangeName': 'OPR',
    'bid': 0.0,
    'ask': 0.05,
    'last': 0.0,
    'mark': 0.03,
    'bidSize': 0,
    'askSize': 100,
    'bidAskSize': '0X100',
    'lastSize': 0,
    'highPrice': 0.0,
    'lowPrice': 0.0,
    'openPrice': 0.0,
    'closePrice': 0.0,
    'totalVolume': 0,
    'tradeDate': None,
    'tradeTimeInLong': 0,
    'quoteTimeInLong': 1600285790589,
    'netChange': 0.0,
    'volatility': 676.61,
    'delta': -0.011,
    'gamma': 0.006,
    'theta': -0.026,
    'vega': 0.0,
    'rho': 0.0,
    'openInterest': 0,
    'timeValue': 0.03,
    'theoreticalOptionValue': 0.025,
    'theoreticalVolatility': 29.0,
    'optionDeliverablesList': None,
    'strikePrice': 2.5,
    'expirationDate': 1600459200000,
    'daysToExpiration': 2,
    'expirationType': 'R',
    'lastTradingDay': 1600473600000,
    'multiplier': 100.0,
    'settlementType': ' ',
    'deliverab

Bueno, como vemos ahi ya tenemos los vencimientos y los contratos, ahora resta recorrer esas listas y meter la data en un frame

In [43]:
v_calls = list(chain['callExpDateMap'].values())
v_calls_fechas = list(chain['callExpDateMap'].keys())
v_puts = list(chain['putExpDateMap'].values())
v_puts_fechas = list(chain['putExpDateMap'].keys())

calls = []
for i in range(len(v_calls)):
    v = list(v_calls[i].values())    
    for j in range(len(v)):
        calls.append(v[j][0])

puts = []
for i in range(len(v_puts)):
    v = list(v_puts[i].values())    
    for j in range(len(v)):
        puts.append(v[j][0])
        
contracts = pd.concat([pd.DataFrame(calls),pd.DataFrame(puts)])
contracts.columns

Index(['putCall', 'symbol', 'description', 'exchangeName', 'bid', 'ask',
       'last', 'mark', 'bidSize', 'askSize', 'bidAskSize', 'lastSize',
       'highPrice', 'lowPrice', 'openPrice', 'closePrice', 'totalVolume',
       'tradeDate', 'tradeTimeInLong', 'quoteTimeInLong', 'netChange',
       'volatility', 'delta', 'gamma', 'theta', 'vega', 'rho', 'openInterest',
       'timeValue', 'theoreticalOptionValue', 'theoreticalVolatility',
       'optionDeliverablesList', 'strikePrice', 'expirationDate',
       'daysToExpiration', 'expirationType', 'lastTradingDay', 'multiplier',
       'settlementType', 'deliverableNote', 'isIndexOption', 'percentChange',
       'markChange', 'markPercentChange', 'inTheMoney', 'nonStandard', 'mini'],
      dtype='object')

In [47]:
contracts.transpose().iloc[:,0]

putCall                                        CALL
symbol                              GGAL_091820C2.5
description               GGAL Sep 18 2020 2.5 Call
exchangeName                                    OPR
bid                                             5.7
ask                                             6.1
last                                              0
mark                                            5.9
bidSize                                          48
askSize                                          48
bidAskSize                                    48X48
lastSize                                          0
highPrice                                         0
lowPrice                                          0
openPrice                                         0
closePrice                                     6.67
totalVolume                                       0
tradeDate                                      None
tradeTimeInLong                                   0
quoteTimeInL

Como la cantidad de columnas es muy grande, armamos una tabla cualquiera para mostrar mas columnas

In [30]:
tabla = contracts.loc[contracts.daysToExpiration > 0]

tabla = tabla.loc[:,['strikePrice','daysToExpiration','putCall','bid','ask',
                     'last','volatility','openInterest',
                     'theoreticalOptionValue']]

tabla.columns = ['Strike','Dias','Tipo','Bid','Ask',
                  'Ultimo','VI','OpenInt','PrimaT']
tabla

,Strike,Dias,Tipo,Bid,Ask,Ultimo,VI,OpenInt,PrimaT
0,2.5,28,CALL,9.80,10.80,0.00,488.089,0,10.300
1,5.0,28,CALL,7.10,8.00,6.30,218.348,3,7.550
2,7.5,28,CALL,4.80,5.90,4.50,186.005,27,5.350
3,10.0,28,CALL,2.55,2.95,3.10,94.005,1522,2.750
4,12.5,28,CALL,0.95,1.05,1.05,74.890,2204,1.000
...,...,...,...,...,...,...,...,...,...
46,12.5,546,PUT,2.25,5.50,4.06,65.255,17,3.875
47,15.0,546,PUT,3.00,7.40,7.53,58.479,17,5.200
48,17.5,546,PUT,5.00,9.40,10.55,60.628,1,7.200
49,20.0,546,PUT,7.00,11.40,9.40,60.788,52,9.200


# Otras

* MarketStack
* Finnhub
* IEX
* FMP
* Tiingo



* ROFEX
* BYMA- IOL
* ALPACA

https://marketstack.com/


https://financialmodelingprep.com/


https://api.tiingo.com/documentation/general/


https://finnhub.io/


https://iexcloud.io/

# Tiingo